In [44]:
import warnings, copy
import pandas as pd
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [55]:
# Load data
df_train = pd.read_csv("raw data/train.csv")
print(df_train.shape)
df_train.head()

(17892, 8)


,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0


In [56]:
df_test = pd.read_csv("raw data/test.csv")
print(df_test.shape)
df_test.head()

(12212, 6)


,ForecastId,Province/State,Country/Region,Lat,Long,Date
0,1,NaN,Afghanistan,33.0,65.0,2020-03-12
1,2,NaN,Afghanistan,33.0,65.0,2020-03-13
2,3,NaN,Afghanistan,33.0,65.0,2020-03-14
3,4,NaN,Afghanistan,33.0,65.0,2020-03-15
4,5,NaN,Afghanistan,33.0,65.0,2020-03-16


In [57]:
# Combine train and test
df = pd.concat([df_train, df_test])
print(df_train.shape, df_test.shape, df.shape)

(17892, 8) (12212, 6) (30104, 9)


In [164]:
# Create a list of alternative country names for better merging
country_names = {
    "Congo (Brazzaville)": ["Congo"],
    "Congo (Kinshasa)": ["Democratic Republic of the Congo"],
    "Czechia": ["Czech Republic"],
    "Gambia, The": ["Gambia"],
    "The Gambia": ["Gambia"],
    "Korea, South": ["South Korea", "Republic of Korea", "Korea"],
    "North Macedonia": ["Republic of North Macedonia"],
    "Republic of the Congo": ["Democratic Republic of the Congo"],
    "Taiwan*": ["Taiwan", "Taiwan Province of China"],
    "Venezuela": ["Bolivarian Republic of Venezuela"],
    "Bolivia": ["Plurinational State of Bolivia"],
    "Brunei": ["Brunei Darussalam"],
    "Cote d'Ivoire": ["Côte d'Ivoire"],
    "Eswatini": ["Swaziland"],
    "Iran": ["Islamic Republic of Iran"],
    "Kosovo": ["Republic of Kosovo"],
    "Moldova": ["Republic of Moldova"],
    "Reunion": ["Réunion"],
    "Russia": ["Russian Federation"],
    "Tanzania": ["United Republic of Tanzania"],
    "The Bahamas": ["Bahamas"],
    "US": ["United States of America", "United States"],
    "United Kingdom": ["United Kingdom of Great Britain and Northern Ireland"],
    "Vietnam": ["Viet Nam"]
}
for country in df['Country/Region'].unique():
    if country in country_names:
        country_names[country].append(country)
    else:
        country_names[country] = [country]

In [63]:
# Combine Country/Region and Province/State to create a new column Place
def func(x):
    try:
        x_new = x['Country/Region'] + "/" + x['Province/State']
    except:
        x_new = x['Country/Region']
    return x_new
        
df['Place'] = df.apply(lambda x: func(x), axis=1)

In [64]:
# Create a new column Day that counts from day 1
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].apply(lambda x: x.dayofyear-21).astype(np.int16)
df.head()

,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,Day
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,1
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,2
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,3
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,4
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,5


In [65]:
# Create new columns Cases/day and Fatals/day
df['Cases/day'] = 0
df['Fatals/day'] = 0
places = np.sort(df['Place'].unique())
for place in places:
    tmp = df[df['Place']==place]['ConfirmedCases'].values
    tmp[1:] -= tmp[:-1]
    df['Cases/day'][df['Place']==place] = tmp
    tmp = df[df['Place']==place]['Fatalities'].values
    tmp[1:] -= tmp[:-1]
    df['Fatals/day'][df['Place']==place] = tmp
print(df.shape)
df[df['Place']=='China/Hubei'].iloc[60:65]

(30104, 13)


,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,Day,Cases/day,Fatals/day
3840,67800.0,China,2020-03-22,3144.0,NaN,5641.0,30.9756,112.2707,Hubei,China/Hubei,61,0,5
3841,67800.0,China,2020-03-23,3153.0,NaN,5642.0,30.9756,112.2707,Hubei,China/Hubei,62,0,9
3842,67801.0,China,2020-03-24,3160.0,NaN,5643.0,30.9756,112.2707,Hubei,China/Hubei,63,1,7
2580,NaN,China,2020-03-12,NaN,2581.0,NaN,30.9756,112.2707,Hubei,China/Hubei,51,-9223372036854775808,-9223372036854775808
2581,NaN,China,2020-03-13,NaN,2582.0,NaN,30.9756,112.2707,Hubei,China/Hubei,52,-9223372036854775808,-9223372036854775808


In [238]:
# Note that Grand Princess and Diamond Princess are assigned twice to two Country/Region
df['Province/State'].value_counts()

Grand Princess      212
Diamond Princess    212
Oregon              106
Inner Mongolia      106
Cayman Islands      106
                   ... 
Mayotte             106
South Australia     106
Yunnan              106
Shanxi              106
Ningxia             106
Name: Province/State, Length: 128, dtype: int64

In [240]:
df[df['Province/State'] == 'Grand Princess']['Country/Region'].unique()

array(['Canada', 'US'], dtype=object)

In [241]:
df[df['Province/State'] == 'Diamond Princess']['Country/Region'].unique()

array(['Cruise Ship', 'US'], dtype=object)

In [68]:
# Create new columns that contain statistics of the past
# Create new columns that tell us how long it has been since 1st/10th/100th case/fatality
def average_interval(df, col, interval):
    df_new = copy.deepcopy(df)
    if interval == [1, 1]:
        col_new = col[:-4] + " yesterday"
    elif interval == [1, 7]:
        col_new = col[:-4] + " last week"
    elif interval == [8, 14]:
        col_new = col[:-4] + " 2weeks ago"
    elif interval == [15, 21]:
        col_new = col[:-4] + " 3weeks ago"
    df_new[col_new] = 0
    tmp = df_new[col].rolling(interval[1]-interval[0]+1).mean()
    df_new[col_new][interval[0]:] = tmp[:-interval[0]]
    df_new[col_new][pd.isna(df_new[col_new])] = 0
    return df_new[[col_new]].reset_index(drop=True)

def add_time_features(df):
    df = pd.concat([df, average_interval(df, 'Cases/day', [1,1]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Cases/day', [1,7]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Cases/day', [8,14]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Cases/day', [15,21]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Fatals/day', [1,1]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Fatals/day', [1,7]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Fatals/day', [8,14]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, average_interval(df, 'Fatals/day', [15,21]).reset_index(drop=True)], axis=1)
    
    for num in [1, 10, 100]:
        days_before = (df['ConfirmedCases']<num).sum()
        tmp = df['Day'].values - 1 - days_before
        tmp[tmp<=0] = 0
        if num == 1:
            col_new = 'Days since 1st case'
        else:
            col_new = 'Days since {}th case'.format(num)
        df[col_new] = tmp
            
    for num in [1, 10, 100]:
        days_before = (df['Fatalities']<num).sum()
        tmp = df['Day'].values - 1 - days_before
        tmp[tmp<=0] = 0
        if num == 1:
            col_new = 'Days since 1st fatal'
        else:
            col_new = 'Days since {}th fatal'.format(num)
        df[col_new] = tmp
    
    # Special handle China/Hubei
    if df['Place'][0]=='China/Hubei':
        df['Days since 1st case'] += 35 # 2019/12/8
        df['Days since 10th case'] += 22 # 2019/12/21
        df['Days since 100th case'] += 4 # 2020/1/18
        df['Days since 1st fatal'] += 13 # 2020/1/9
    return df

In [69]:
import copy
df2 = []
for place in places[:]:
    df_tmp = df[df['Place']==place].reset_index(drop=True)
    df_tmp = add_time_features(df_tmp)
    df2.append(df_tmp)
df2 = pd.concat(df2).reset_index(drop=True)
df2[df2['Place']=='China/Hubei'].head()

,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,Fatals yesterday,Fatals last week,Fatals 2weeks ago,Fatals 3weeks ago,Days since 1st case,Days since 10th case,Days since 100th case,Days since 1st fatal,Days since 10th fatal,Days since 100th fatal
6360,444.0,China,2020-01-22,17.0,NaN,5581.0,30.9756,112.2707,Hubei,China/Hubei,...,0.0,0.0,0.0,0.0,35,22,4,13,0,0
6361,444.0,China,2020-01-23,17.0,NaN,5582.0,30.9756,112.2707,Hubei,China/Hubei,...,17.0,0.0,0.0,0.0,36,23,5,14,1,0
6362,549.0,China,2020-01-24,24.0,NaN,5583.0,30.9756,112.2707,Hubei,China/Hubei,...,0.0,0.0,0.0,0.0,37,24,6,15,2,0
6363,761.0,China,2020-01-25,40.0,NaN,5584.0,30.9756,112.2707,Hubei,China/Hubei,...,7.0,0.0,0.0,0.0,38,25,7,16,3,0
6364,1058.0,China,2020-01-26,52.0,NaN,5585.0,30.9756,112.2707,Hubei,China/Hubei,...,16.0,0.0,0.0,0.0,39,26,8,17,4,0


In [71]:
# Load smoking rate data (source: https://ourworldindata.org/smoking)
df_smoking = pd.read_csv("raw data/share-of-adults-who-smoke.csv")
print(df_smoking.shape)
df_smoking.head()

(1680, 4)


,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Albania,ALB,2000,34.8
1,Albania,ALB,2005,32.7
2,Albania,ALB,2010,31.2
3,Albania,ALB,2011,30.7
4,Albania,ALB,2012,30.2


In [72]:
# Get most updated smoking rate for each country
df_smoking_new = df_smoking.sort_values('Year', ascending=False).reset_index(drop=True)
df_smoking_new = df_smoking_new[df_smoking_new['Entity'].duplicated()==False]
print(df_smoking_new.shape)
df_smoking_new.head()

(187, 4)


,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Zimbabwe,ZWE,2016,15.8
1,Namibia,NAM,2016,21.4
2,Suriname,SUR,2016,25.0
3,Morocco,MAR,2016,23.4
4,India,IND,2016,11.5


In [79]:
# Merge with main table to get a new column "Smoking rate"
entities = set(df_smoking_new['Entity'].values)
tmp = []
for country in df2['Country/Region'].values:
    found = False
    for name in country_names[country]:
        if name in entities:
            tmp.append(df_smoking_new[df_smoking_new['Entity']==name]['Smoking prevalence, total (ages 15+) (% of adults)'].values[0])
            found = True
            break
    if not found:
        tmp.append(NaN)
print(len(tmp))
df2['Smoking rate'] = tmp
df2.head()

30104


,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,Fatals last week,Fatals 2weeks ago,Fatals 3weeks ago,Days since 1st case,Days since 10th case,Days since 100th case,Days since 1st fatal,Days since 10th fatal,Days since 100th fatal,Smoking rate
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,NaN
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,NaN
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,NaN
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,NaN
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,NaN


In [85]:
# Use world smoking rate for countries with no smoking rate data
world_rate = df_smoking_new[df_smoking_new['Entity']=='World']['Smoking prevalence, total (ages 15+) (% of adults)'].values[0]
print(world_rate)
df2['Smoking rate'][pd.isna(df2['Smoking rate'])] = world_rate
df2.head()

20.4755174405242


,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,Fatals last week,Fatals 2weeks ago,Fatals 3weeks ago,Days since 1st case,Days since 10th case,Days since 100th case,Days since 1st fatal,Days since 10th fatal,Days since 100th fatal,Smoking rate
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,20.475517
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,...,0.0,0.0,0.0,0,0,0,0,0,0,20.475517


In [95]:
# Load GDP/employment/population data (source: https://www.imf.org/external/pubs/ft/weo/2019/02/weodata/index.aspx)
df_weo = pd.read_csv("raw data/WEO_Data.csv", encoding = "ISO-8859-1")
print(df_weo.shape)
df_weo.head()

(1164, 8)


,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2019,2020,Estimates Start After
0,Afghanistan,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",18.734,18.861,2018.0
1,Afghanistan,"Gross domestic product, current prices",Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",76.486,80.717,2018.0
2,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",513.108,509.759,2016.0
3,Afghanistan,Gross domestic product based on purchasing-pow...,Percent,NaN,"See notes for: Gross domestic product, curren...",0.054,0.054,2018.0
4,Afghanistan,Unemployment rate,Percent of total labor force,NaN,NaN,NaN,NaN,NaN


In [111]:
# Combine Subject Descriptor and Unit into a new column called Subject
tmp = []
for i in range(len(df_weo)):
    tmp.append(df_weo.iloc[i].values[1] + ' (' + df_weo.iloc[i].values[2] + ')')
df_weo['Subject'] = tmp
print(df_weo.shape)
df_weo.head()

(1164, 9)


,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2019,2020,Estimates Start After,Subject
0,Afghanistan,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",18.734,18.861,2018.0,"Gross domestic product, current prices (U.S. d..."
1,Afghanistan,"Gross domestic product, current prices",Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",76.486,80.717,2018.0,"Gross domestic product, current prices (Purcha..."
2,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",513.108,509.759,2016.0,"Gross domestic product per capita, current pri..."
3,Afghanistan,Gross domestic product based on purchasing-pow...,Percent,NaN,"See notes for: Gross domestic product, curren...",0.054,0.054,2018.0,Gross domestic product based on purchasing-pow...
4,Afghanistan,Unemployment rate,Percent of total labor force,NaN,NaN,NaN,NaN,NaN,Unemployment rate (Percent of total labor force)


In [112]:
df_weo['Subject'].unique()

array(['Gross domestic product, current prices (U.S. dollars)',
       'Gross domestic product, current prices (Purchasing power parity; international dollars)',
       'Gross domestic product per capita, current prices (U.S. dollars)',
       'Gross domestic product based on purchasing-power-parity (PPP) share of world total (Percent)',
       'Unemployment rate (Percent of total labor force)',
       'Population (Persons)'], dtype=object)

In [196]:
# Transpose the table to make each subject a column
subjects = df_weo['Subject'].unique()
df_weo_t = df_weo[['Country']][df_weo['Country'].duplicated()==False].reset_index(drop=True)
for subject in subjects:
    df_tmp = df_weo[['Country', '2019']][df_weo['Subject']==subject].reset_index(drop=True)
    df_tmp = df_tmp[df_tmp['Country'].duplicated()==False].reset_index(drop=True)
    df_tmp.columns = ['Country', subject]
    df_weo_t = df_weo_t.merge(df_tmp, on='Country', how='left')
df_weo_t.columns = ['Country', 'Nominal GDP', 'GDP based on PPP', 'GDP per capita', 'GDP based on PPP share of world total', 'Unemployment rate', 'Population']
print(df_weo_t.shape)
df_weo_t.head()

(194, 7)


,Country,Nominal GDP,GDP based on PPP,GDP per capita,GDP based on PPP share of world total,Unemployment rate,Population
0,Afghanistan,18.734,76.486,513.108,0.054,NaN,36.51
1,Albania,15.418,40.151,"5,372.74",0.028,13.719,2.87
2,Algeria,172.781,681.396,"3,980.12",0.48,12.472,43.411
3,Angola,91.527,203.433,"3,037.98",0.143,NaN,30.128
4,Antigua and Barbuda,1.688,2.735,"18,109.10",0.002,NaN,0.093


In [214]:
def merge_tables(t1, t2, col, entities):
    tmp = []
    for country in t1['Country/Region'].values:
        found = False
        for name in country_names[country]:
            if name in entities:
                tmp.append(t2[t2['Country']==name][col].values[0])
                found = True
                break
        if not found:
            tmp.append(NaN)
    return tmp

In [137]:
# Merge with main table to get new columns about GDP, unemployment rate, and population
entities = set(df_weo_t['Country'].values)
for col in df_weo_t.columns[1:]:
    tmp = merge_tables(df2, df_weo_t, col, entities)
    df2[col] = tmp
print(df2.shape)
df2.head()

(30104, 34)


,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,Days since 1st fatal,Days since 10th fatal,Days since 100th fatal,Smoking rate,Nominal GDP,GDP based on PPP,GDP per capita,GDP based on PPP share of world total,Unemployment rate,Population
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,...,0,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,...,0,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,...,0,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,...,0,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,...,0,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51


In [220]:
# Load life expectancy data (source: http://hdr.undp.org/en/data)
df_life = pd.read_csv("raw data/Life expectancy at birth.csv")
tmp = df_life.iloc[:,1].values.tolist()
df_life = df_life[['Country', '2018']]
print(df_life.shape)
df_life.head()

(209, 2)


,Country,2018
0,Afghanistan,64.5
1,Albania,78.5
2,Algeria,76.7
3,Andorra,81.8
4,Angola,60.8


In [221]:
# Merge with main table to get new columns about GDP, unemployment rate, and population
df_life.columns = ['Country', 'Life expectancy']
entities = set(df_life['Country'].values)
tmp = merge_tables(df2, df_life, 'Life expectancy', entities)
df2['Life expectancy'] = tmp
print(df2.shape)
df2.head()

(30104, 35)


,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,Days since 10th fatal,Days since 100th fatal,Smoking rate,Nominal GDP,GDP based on PPP,GDP per capita,GDP based on PPP share of world total,Unemployment rate,Population,Life expectancy
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5


In [223]:
# Load data that contains additional information per country (source: https://www.kaggle.com/koryto/countryinfo)
df_info = pd.read_csv("raw data/covid19countryinfo.csv")
df_info.head()

,country,pop,tests,testpop,density,medianage,urbanpop,quarantine,schools,restrictions,...,sex0,sex14,sex25,sex54,sex64,sex65plus,sexratio,lung,femalelung,malelung
0,Afghanistan,"38,928,346",NaN,NaN,60.0,18.0,25.0,NaN,NaN,NaN,...,1.05,1.03,1.03,1.03,0.97,0.85,1.03,37.62,36.31,39.33
1,Albania,"2,877,797",NaN,NaN,105.0,36.0,63.0,NaN,NaN,NaN,...,1.08,1.11,1.09,0.93,0.95,0.87,0.98,11.67,7.02,17.04
2,Algeria,"43,851,044",NaN,NaN,18.0,29.0,73.0,NaN,NaN,NaN,...,1.05,1.05,1.05,1.03,1.01,0.89,1.03,8.77,5.03,12.81
3,Andorra,"77,265",NaN,NaN,164.0,45.0,88.0,NaN,NaN,NaN,...,1.07,1.05,1.08,1.05,1.15,1.02,1.06,NaN,NaN,NaN
4,Antigua and Barbuda,"97,929",NaN,NaN,223.0,34.0,26.0,NaN,NaN,NaN,...,1.05,1.03,0.99,0.84,0.82,0.76,0.90,11.76,7.67,18.78


In [224]:
# Count missing values
df_info.isnull().sum()

country           0
pop              55
tests           167
testpop         167
density          55
medianage        55
urbanpop         55
quarantine      191
schools         155
restrictions    169
hospibed         55
smokers         111
sex0             61
sex14            61
sex25            61
sex54            61
sex64            61
sex65plus        61
sexratio         62
lung             74
femalelung       75
malelung         76
dtype: int64

In [228]:
# Ignore columns with too many missing values and those that seem highly correlated
# Rename columns
df_info = df_info[['country', 'density', 'medianage', 'urbanpop', 'hospibed',
                   'sexratio', 'lung']]
df_info.columns = ['Country', 'Population density', 'Median age', 'Urban population %', 'Hospital beds',
                   'Sex ratio', 'Death rate from lung diseases']

In [234]:
df2.head()

,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,Days since 10th fatal,Days since 100th fatal,Smoking rate,Nominal GDP,GDP based on PPP,GDP per capita,GDP based on PPP share of world total,Unemployment rate,Population,Life expectancy
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,...,0,0,20.475517,18.734,76.486,513.108,0.054,NaN,36.51,64.5


In [235]:
# Merge with main table to get new columns of additional information
entities = set(df_info['Country'].values)
for col in df_info.columns[1:]:
    tmp = merge_tables(df2, df_info, col, entities)
    df2[col] = tmp
print(df2.shape)
df2.head()

(30104, 41)


,ConfirmedCases,Country/Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province/State,Place,...,GDP based on PPP share of world total,Unemployment rate,Population,Life expectancy,Population density,Median age,Urban population %,Hospital beds,Sex ratio,Death rate from lung diseases
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,33.0,65.0,NaN,Afghanistan,...,0.054,NaN,36.51,64.5,60.0,18.0,25.0,0.5,1.03,37.62
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,33.0,65.0,NaN,Afghanistan,...,0.054,NaN,36.51,64.5,60.0,18.0,25.0,0.5,1.03,37.62
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,33.0,65.0,NaN,Afghanistan,...,0.054,NaN,36.51,64.5,60.0,18.0,25.0,0.5,1.03,37.62
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,33.0,65.0,NaN,Afghanistan,...,0.054,NaN,36.51,64.5,60.0,18.0,25.0,0.5,1.03,37.62
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,33.0,65.0,NaN,Afghanistan,...,0.054,NaN,36.51,64.5,60.0,18.0,25.0,0.5,1.03,37.62


In [245]:
df2.dtypes

ConfirmedCases                                  float64
Country/Region                                   object
Date                                     datetime64[ns]
Fatalities                                      float64
ForecastId                                      float64
Id                                              float64
Lat                                             float64
Long                                            float64
Province/State                                   object
Place                                            object
Day                                               int16
Cases/day                                         int64
Fatals/day                                        int64
Cases yesterday                                 float64
Cases last week                                 float64
Cases 2weeks ago                                float64
Cases 3weeks ago                                float64
Fatals yesterday                                

In [247]:
# Convert multiple columns to float
def to_float(x):
    x_new = 0
    try:
        x_new = float(x.replace(",", ""))
    except:
        x_new = np.nan
    return x_new

float_cols = [
    'Cases/day',
    'Fatals/day',
    'Nominal GDP',
    'GDP based on PPP',
    'GDP per capita',
    'GDP based on PPP share of world total',
    'Unemployment rate',
    'Population',
    'Life expectancy'
]

for col in float_cols:
    df2[col] = df2[col].apply(lambda x: to_float(x))  

In [248]:
df2.dtypes

ConfirmedCases                                  float64
Country/Region                                   object
Date                                     datetime64[ns]
Fatalities                                      float64
ForecastId                                      float64
Id                                              float64
Lat                                             float64
Long                                            float64
Province/State                                   object
Place                                            object
Day                                               int16
Cases/day                                       float64
Fatals/day                                      float64
Cases yesterday                                 float64
Cases last week                                 float64
Cases 2weeks ago                                float64
Cases 3weeks ago                                float64
Fatals yesterday                                

In [250]:
# Separate train/test and save the transformed datasets
# Note that training period is from day 1 to day 63
# Test period is from day 51 to day 93, but we're only using day 64 to day 93
df_train = df2[(pd.isna(df2['ForecastId'])) & (df2['Day']<=63)]
df_test = df2[(pd.isna(df2['ForecastId'])==False) & (df2['Day']>=64)]

In [253]:
df_train.to_csv('data/train.csv', index=None)
df_test.to_csv('data/test.csv', index=None)